# Sparkify - Predicting User Churn

### 1. PySpark Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import findspark
findspark.init()

In [2]:
spark = SparkSession \
    .builder \
    .appName("Sparkify Churn Prediction") \
    .getOrCreate()

## 2. Load and Clean Dataset

In [5]:
#Read event data from mini file
path = "data/mini_sparkify_event_data.json"
sparkify_event_log = spark.read.json(path)

#Create a view for running SQL queries
sparkify_event_log.createOrReplaceTempView("log_table")


In [6]:
#Display the data schema to see what data is available about user events
sparkify_event_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
#View first 5 rows of the log
sparkify_event_log.take(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [17]:
#Check for any rows with a missing sessionId - these would likely be a data error within the Sparkify system
spark.sql('''
          SELECT count(sessionId)
          FROM log_table 
          WHERE sessionId == ''
          '''
          ).show()
#There are no rows which are missing a sessionId so no cleaning need to address these

+----------------+
|count(sessionId)|
+----------------+
|               0|
+----------------+



In [15]:
#Check for any rows with a missing userId - these won't be useful to predict user churn
spark.sql('''
          SELECT count(userId)
          FROM log_table 
          WHERE userId == ''
          '''
          ).show()

spark.sql('''
          SELECT DISTINCT auth, page
          FROM log_table
          WHERE userId == ''
          '''
         ).show()
#There are many rows without a userId
#Looking at the events these relate to, they relate to Sparkify information pages such as Home, About and Help
#It makes sense that these would be accessed by logged out or guest users and can be dropped from the dataset as not useful

+-------------+
|count(userId)|
+-------------+
|         8346|
+-------------+

+----------+-------------------+
|      auth|               page|
+----------+-------------------+
|Logged Out|               Home|
|     Guest|           Register|
|Logged Out|              Error|
|     Guest|              About|
|     Guest|              Error|
|Logged Out|              Login|
|Logged Out|               Help|
|     Guest|Submit Registration|
|Logged Out|              About|
|     Guest|               Help|
|     Guest|               Home|
+----------+-------------------+



In [38]:
#Select only rows that have a userId attached
#Only keep columns that look useful for further investigation - can drop names, userAgent string, status, location etc.
user_event_log = spark.sql('''
                            SELECT artist, gender, itemInSession, length, level, page, sessionId, song, ts, userId
                            FROM log_table
                            WHERE userId != ''
                            '''
                          )
#Create view for this new table
user_event_log.createOrReplaceTempView("user_log_table")

## 3. Exploratory Data Analysis

In [43]:
#Investigate page column
page_counts = spark.sql("SELECT page, COUNT(page) AS page_counts \
        FROM log_table \
        GROUP BY page").show(30)

+--------------------+-----------+
|                page|page_counts|
+--------------------+-----------+
|              Cancel|         52|
|    Submit Downgrade|         63|
|         Thumbs Down|       2546|
|                Home|      14457|
|           Downgrade|       2055|
|         Roll Advert|       3933|
|              Logout|       3226|
|       Save Settings|        310|
|Cancellation Conf...|         52|
|               About|        924|
| Submit Registration|          5|
|            Settings|       1514|
|               Login|       3241|
|            Register|         18|
|     Add to Playlist|       6526|
|          Add Friend|       4277|
|            NextSong|     228108|
|           Thumbs Up|      12551|
|                Help|       1726|
|             Upgrade|        499|
|               Error|        258|
|      Submit Upgrade|        159|
+--------------------+-----------+



In [ ]:
#Create CHURN flag

## 4. Feature Engineering

## 5. Modelling